# Introduction
In this practical, a branch predictor simulator was written which simulates different branching strategies to get their misprediction rates. Then, the results were analysed and presented. The following branch prediction strategies were simulated and analysed:
- Always taken
- Always not taken
- 2-bit predictor
- correlating predictor
- gshare
- profiled approach

## Profiled approach
The profile approach is a strategy that uses a previously run "profile" of the code for its prediction. Details of the profile approach used will be explained [later](#profiled_approach) in the report.

# Design

## Basic classes
First, let us define some useful classes that we will use in our simulator.

In [1]:
from enum import Enum

class Branch(Enum):
    NOT_TAKEN = 0
    TAKEN = 1

This `Branch` enum represents whether the branch was taken or not. An enum is used instead of 0 and 1 to make it clear and avoid mistakes and errors.

Now we create a class to represent each step of the execution:

In [2]:
class Step:
    def __init__(self, address, branch):
        self.address = address
        self.branch = branch

`Step` contains the address of the instruction as well as whether or not that branch was taken - stored in `self.branch` - for each step of the execution. This is the class that the output of the `pin` tool will parse into.

Now we add an abstract class for branch prediction strategies that the actual strategy implementations will use. This just ensures that all implemented strategies later always have a default prediction and `update` function. A `Strategy` is the basic saturation counter that the branch history table points to.

In [3]:
from abc import ABC, abstractmethod

class Strategy(ABC):
    def __init__(self):
        self.prediction = Branch.NOT_TAKEN
    
    @abstractmethod
    def update(self, prediction_correct):
        pass

The default prediction is `NOT_TAKEN` and we define an abstract method `update` that all strategy implementations must have. The `update` method is where the strategy updates its own calculations and prediction based whether the previous prediction was correct or not.

Finally we have the `Predictor` which takes any strategy and steps of execution and runs the strategy over those executions. 

In [4]:
class Predictor:
    def __init__(self, strategy, execution):
        self.strategy = strategy
        self.execution = execution
        
    def predict(self, step):
        prediction = self.strategy.prediction
        
        prediction_correct = step.branch == prediction
        self.strategy.update(prediction_correct)
        
        return prediction_correct
    
    def simulate(self):
        correct_prediction_count = 0
        
        for step in self.execution:
            if self.predict(step):
                correct_prediction_count += 1
            
        return correct_prediction_count / len(self.execution) * 100

This practical is focused on the results of simulating different branch prediction strategies, so the `simulate` function will run the `Predictor`'s strategy over whole execution and return the percentage of correct predictions. On each `predict` step, the prediction strategy is updated with whether or not it was correct.

Now that we have all the basic classes set up, we can start implementing the two simplest branch prediction strategies.

## Always taken and Always not taken

Always taken and always not taken are very easy to implement because they do not use any smart logic. They simply always take or not take the branch.

In [5]:
class AlwaysTaken(Strategy):
    def __init__(self):
        self.prediction = Branch.TAKEN
        
    def update(self, _):
        pass


class AlwaysNotTaken(Strategy):
    
    def update(self, _):
        pass

Both strategies do not require any update logic as they always return the same prediction. In `AlwaysTaken`, the prediction is set to be `TAKEN`. We don't have to do this for `AlwaysNotTaken` as all strategies default to `NOT_TAKEN`.

## Two-bit predictor

The two-bit predictor uses two bits as history bits. The prediction is based off the history bits and they are updated on every step.

In [7]:
class TwoBitPredictor(Strategy):
    def __init__(self):
        super().__init__()
        self.history_bits = (0, 0)
        
    def update(self, prediction_correct):
        self.update_bits(prediction_correct)
        
        if self.history_bits[0]:
            self.prediction = Branch.TAKEN
        else:
            self.prediction = Branch.NOT_TAKEN
            
    def update_bits(self, prediction_correct):
        prediction_value = self.prediction.value
        
        if (prediction_correct):
            self.history_bits = (prediction_value, prediction_value)
        else:
            last_bit = self.history_bits[1]
            self.history_bits = (last_bit, int(not prediction_value))
            

## Profiled approach <a id='profiled_approach'></a>